In [5]:
from qiskit.circuit import Parameter
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
import numpy as np
from scipy.optimize import minimize
import networkx as nx

def qaoa_ansatz(num_qubits, p):
    
    graph = nx.Graph()
    graph.add_edges_from([(i, (i + 1) % num_qubits) for i in range(num_qubits)])  # Circular graph

    qc = QuantumCircuit(num_qubits)

    # Create parameters for the ansatz
    beta = [Parameter(f'β{i}') for i in range(p)]  # Mixer angles
    gamma = [Parameter(f'γ{i}') for i in range(p)]  # Cost angles

    # Initial state: |+> state on all qubits
    qc.h(range(num_qubits))

    # QAOA layers
    for i in range(p):
        # Apply the cost Hamiltonian (based on graph edges)
        for u, v in graph.edges:
            qc.rzz(2 * gamma[i], u, v)
        
        # Apply the mixer Hamiltonian (X rotations)
        qc.rx(2 * beta[i], range(num_qubits))

    return qc, beta + gamma

def get_molecule_hamiltonian(driver):
    es_problem = driver.run()
    second_q_op = es_problem.second_q_ops()[0]
    mapper = JordanWignerMapper()
    qubit_hamiltonian = mapper.map(second_q_op)
    return qubit_hamiltonian

def qaoa_ansatz(num_qubits, p):
    graph = nx.Graph()
    graph.add_edges_from([(i, (i + 1) % num_qubits) for i in range(num_qubits)])
    qc = QuantumCircuit(num_qubits)
    beta = [Parameter(f'β{i}') for i in range(p)]
    gamma = [Parameter(f'γ{i}') for i in range(p)]
    qc.h(range(num_qubits))
    for i in range(p):
        for u, v in graph.edges:
            qc.rzz(2 * gamma[i], u, v)
        qc.rx(2 * beta[i], range(num_qubits))
    return qc, beta + gamma

def cost_function(params, hamiltonian, num_qubits):
    p = len(params) // 2  # p is the number of layers
    qc, param_vec = qaoa_ansatz(num_qubits, p)
    param_dict = {param_vec[i]: params[i] for i in range(len(param_vec))}
    bound_circuit = qc.assign_parameters(param_dict)
    estimator = Estimator()
    result = estimator.run(bound_circuit, [hamiltonian])
    expectation_value = result.result().values[0]
    return expectation_value

def optimization_loop(hamiltonian, num_qubits, optimizer, initial_params):
    def cost(params):
        return cost_function(params, hamiltonian, num_qubits)
    result = minimize(cost, initial_params, method=optimizer, options={'maxiter': 1000, 'tol': 1e-6})
    return result.fun, result.x

driver = PySCFDriver(
    atom="Li 0 0 0; H 0 0 1.5949",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

qubit_hamiltonian = get_molecule_hamiltonian(driver)
num_qubits = qubit_hamiltonian.num_qubits

# Generate a random initial set of parameters
p = 1 # Number of QAOA layers
initialization_params = np.random.uniform(0, 2 * np.pi, 2 * p)
#initialization_params = np.full(num_qubits, 0)

#gamma_init = np.full(p, np.pi*(3/2))  # Initial values for gamma parameters
#beta_init = np.full(p, np.pi*(0))   # Initial values for beta parameters

# Combine them into a single parameter array for optimization
#initialization_params = np.concatenate([gamma_init, beta_init])

optimizer = 'COBYLA'

def create_ansatz_circuit():
    return qaoa_ansatz(num_qubits, p)[0]


# Perform the optimization manually
optimal_value, optimal_params = optimization_loop(qubit_hamiltonian, num_qubits, optimizer, initialization_params)
print(f'Optimal energy: {optimal_value} Hartree')


Optimal energy: -5.464826085507828 Hartree
